## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>


</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#Intall beautifulsoup4 & lxml
!conda install -c conda-forge beautifulsoup4 --yes 
!conda install -c conda-forge lxml --yes 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/shanshanjin/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

**1.1 Sydney house prices of 2019**

Read data

In [2]:
df_hp=pd.read_csv('SydneyHousePrices.csv')

In [3]:
df_hp['suburb']=df_hp['suburb'].str.strip()
df_hp['Year']=df_hp['Date'].str[:4]

Select 2019 housing price and average by suburb

In [4]:
df_hp=df_hp[(df_hp['Year']=='2019')& (df_hp['propType']=='house' )]
df_hp=df_hp.groupby(['suburb','postalCode'])[['sellPrice']].mean().round(2)
df_hp.reset_index(inplace=True)


Normalize SellPrice

In [5]:
x=df_hp[['sellPrice']].values
normalized_x=(x-x.mean())/x.std()
df_hp['SDPrice']= pd.DataFrame(normalized_x)
df_hp.head()

,suburb,postalCode,sellPrice,SDPrice
0,Abbotsbury,2176,890090.91,-0.496483
1,Abbotsford,2046,2490000.00,1.081061
2,Agnes Banks,2753,745000.00,-0.639546
3,Airds,2560,410461.54,-0.969408
4,Alexandria,2015,1446750.00,0.052395


This step get the standarded average sellPrice for all suburbs.

In [6]:
df_hp.shape

(570, 4)

-----

**1.2 Got OC list**

In [7]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import pandas as pd
# get table from url
source = urllib.request.urlopen("https://education.nsw.gov.au/public-schools/selective-high-schools-and-opportunity-classes/year-5/what-are-opportunity-classes/list-of-opportunity-classes").read()
soup = BeautifulSoup(source,'lxml')
table = soup.find('table', attrs={'id':"table45380"})


table_rows = table.find_all('tr')

#extract rows from table and append them into a dataframe
l=[]
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td] 
    l.append(row)
df_oc=pd.DataFrame(l[1:], columns=["Code", "SL"])
df_oc.head()

,Code,SL
0,ACP,"Armidale City Faulkner St, Armidale"
1,ALX,"Alexandria Park Park Rd, Alexandria"
2,ALS,"Alstonville Main St, Alstonville"
3,ART,"Artarmon McMillan Rd, Artarmon"
4,AUR,"Aurora Rural and remote, virtual opportunity c..."


In [8]:
df_oc[['School','suburb']] = df_oc['SL'].str.rsplit(',',n=1, expand=True)
df_oc['suburb']=df_oc['suburb'].str.strip()
df_oc=df_oc.groupby(['suburb'])[['Code']].count().reset_index()
df_oc.head()

,suburb,Code
0,Alexandria,1
1,Alstonville,1
2,Armidale,1
3,Artarmon,1
4,Ashfield,1


In [9]:
#This step add the oc school number to suburbs.
df_hp=pd.merge(df_hp,df_oc.drop_duplicates(),on='suburb',how='left')
df_hp.fillna(0,inplace=True)
df_hp.rename(columns={'Code':'CountOfOC'},inplace=True)
df_hp.head()

,suburb,postalCode,sellPrice,SDPrice,CountOfOC
0,Abbotsbury,2176,890090.91,-0.496483,0.0
1,Abbotsford,2046,2490000.00,1.081061,0.0
2,Agnes Banks,2753,745000.00,-0.639546,0.0
3,Airds,2560,410461.54,-0.969408,0.0
4,Alexandria,2015,1446750.00,0.052395,1.0


In [22]:
df_hp.shape

(570, 5)



------


**1.3 Get Geo Data**

In [10]:
df_geo=pd.read_csv('australian_postcodes.csv')

In [11]:
df_geo=df_geo[df_geo['state']=='NSW']

In [12]:
df_geo=df_geo[['long','lat','sa3name','locality']]
df_geo.rename(columns={'locality':'suburb'},inplace=True)
df_geo['suburb']=df_geo['suburb'].apply(lambda x: x.title())
df_geo.drop_duplicates(['long','lat','suburb'],inplace=True)
df_geo.head()

,long,lat,sa3name,suburb
420,151.268071,-33.794883,Sydney Inner City,Sydney
441,150.874182,-33.662834,Sydney Inner City,Sydney
468,150.866145,-33.666729,Sydney Inner City,Sydney
469,150.870220,-33.664575,Sydney Inner City,Sydney
484,150.874265,-33.662790,Sydney Inner City,Sydney


Merge geo info to df_hp

In [13]:
sydney_suburb=pd.merge(df_hp,df_geo,on='suburb',how='left')
sydney_suburb.head()

,suburb,postalCode,sellPrice,SDPrice,CountOfOC,long,lat,sa3name
0,Abbotsbury,2176,890090.91,-0.496483,0.0,150.885260,-33.872014,Fairfield
1,Abbotsford,2046,2490000.00,1.081061,0.0,151.133865,-33.866044,Strathfield - Burwood - Ashfield
2,Agnes Banks,2753,745000.00,-0.639546,0.0,150.646053,-33.604022,Richmond - Windsor
3,Airds,2560,410461.54,-0.969408,0.0,150.768408,-34.194216,Wollondilly
4,Alexandria,2015,1446750.00,0.052395,1.0,151.108248,-33.711785,Sydney Inner City


In [15]:
#rename columns
sydney_suburb.rename(columns={'suburb':'Neighborhood','long':'Longitude','lat':'Latitude','sa3name':'Borough','sellPrice':'SellPrice'},inplace=True)
sydney_suburb=sydney_suburb[sydney_suburb['Latitude']!=0]
sydney_suburb.dropna(inplace=True)
sydney_suburb.head()

,Neighborhood,postalCode,SellPrice,SDPrice,CountOfOC,Longitude,Latitude,Borough
0,Abbotsbury,2176,890090.91,-0.496483,0.0,150.885260,-33.872014,Fairfield
1,Abbotsford,2046,2490000.00,1.081061,0.0,151.133865,-33.866044,Strathfield - Burwood - Ashfield
2,Agnes Banks,2753,745000.00,-0.639546,0.0,150.646053,-33.604022,Richmond - Windsor
3,Airds,2560,410461.54,-0.969408,0.0,150.768408,-34.194216,Wollondilly
4,Alexandria,2015,1446750.00,0.052395,1.0,151.108248,-33.711785,Sydney Inner City


In [37]:
#rename columns
# df_hp.rename(columns={'suburb':'Neighborhood','sellPrice':'SellPrice'},inplace=True)

In [36]:
# sydney_suburb.shape

In [27]:
# sydney_suburb[sydney_suburb['postalCode']==2121]

In [28]:
# df_geo[df_geo['suburb']=='Epping']

------

**1.4 Finish Cleaning and Generate DF that would be used next**

In [35]:
sydney_suburb.head()# Main table contain all basic information

,Neighborhood,postalCode,SellPrice,SDPrice,CountOfOC,Longitude,Latitude,Borough
0,Abbotsbury,2176,890090.91,-0.496483,0.0,150.885260,-33.872014,Fairfield
1,Abbotsford,2046,2490000.00,1.081061,0.0,151.133865,-33.866044,Strathfield - Burwood - Ashfield
2,Agnes Banks,2753,745000.00,-0.639546,0.0,150.646053,-33.604022,Richmond - Windsor
3,Airds,2560,410461.54,-0.969408,0.0,150.768408,-34.194216,Wollondilly
4,Alexandria,2015,1446750.00,0.052395,1.0,151.108248,-33.711785,Sydney Inner City


In [29]:
neighborhoods= sydney_suburb[['Neighborhood','Longitude','Latitude','Borough']]#information for foursquare API

In [30]:
neighborhoods.head()

,Neighborhood,Longitude,Latitude,Borough
0,Abbotsbury,150.885260,-33.872014,Fairfield
1,Abbotsford,151.133865,-33.866044,Strathfield - Burwood - Ashfield
2,Agnes Banks,150.646053,-33.604022,Richmond - Windsor
3,Airds,150.768408,-34.194216,Wollondilly
4,Alexandria,151.108248,-33.711785,Sydney Inner City


Have a look how many boroughs and neighborhoods we get

In [31]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 51 boroughs and 632 neighborhoods.


**1.5 Use geopy library to get the latitude and longitude values of Sydney City.**

In [33]:
address = 'Sydney NSW,Australia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sydney City are -33.8548157, 151.2164539.


-----